# Load dataframe from Wikipedia

In [1]:
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)

df = df.groupby('PostalCode', as_index=False).agg({
    "Borough" : 'first',
    "Neighborhood": lambda x: ', '.join(x)    
})

for index, row in df.iterrows():
    if df.loc[index, 'Neighborhood'] == 'Not assigned':
        df.loc[index, 'Neighborhood'] = df.loc[index, 'Borough']

df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


# Get the latitude and the longitude coordinates of each neighborhood

In [2]:
dfGeo = pd.read_csv('http://cocl.us/Geospatial_data')
dfGeo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# Merge df and dfGeo in the dataframe

In [3]:
dfToronto = pd.merge(left=df, right=dfGeo, how='left', left_on='PostalCode', right_on='Postal Code')
dfToronto.drop(labels='Postal Code', axis=1, inplace=True)
dfToronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
